In [1]:
import pandas as pd
from datetime import datetime
import os
import numpy as np
import pymongo

In [ ]:
appended_data2020 = []
path = os.path.join("Original_Files","CountyHealthRankingsData","2020")
for WorkingFile in os.listdir(path):
        #identifies the list of files to be grouped together from designated folder
        print(WorkingFile)
        
        # Import the excel file by joining the path directory and file name together and call Ranked Data and Additonal Measure tabs
        file = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Ranked Measure Data',header=(0,1))
        file2 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Additional Measure Data',header=(0,1))
       
        # Load the excel files as two dataframes
        df = pd.DataFrame(file)
        df2 = pd.DataFrame(file2)
        
        # Code from Data_cleaning
       
        # Clean Ranked Data First by addressing multiindex column names
        df.columns = df.columns.map('_'.join)
        
        # Rename Columns
        df.rename(columns = {'Unnamed: 0_level_0_FIPS' : 'FIPS',
         'Unnamed: 1_level_0_State' :'State',
         'Unnamed: 2_level_0_County':'County/Borough/Parish',
         'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
         'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
         'Premature death_Deaths':'Premature_Deaths',
         'Premature death_Years of Potential Life Lost Rate':'Yrs_Potential_Life_Lost_per100K(YPLL)',
         'Poor or fair health_% Fair or Poor Health':'%_Fair/Poor_Health',
         'Low birthweight_% Low Birthweight':'%_Low_Birthweight_Births',
         'Adult smoking_% Smokers':'%_Adult_Smokers',
         'Adult obesity_% Adults with Obesity':'%_Adult_Obesity',
         'Food environment index_Food Environment Index':'Healthy_Food_Access(0=Worst,10=Best)',
         'Physical inactivity_% Physically Inactive':'%_Adults_Physical_Inactivity',
         'Access to exercise opportunities_% With Access to Exercise Opportunities':'%_Access_to_Excercise_Opportunities',
         'Excessive drinking_% Excessive Drinking':'%_Adults_Excessive_Drinking',
         'Alcohol-impaired driving deaths_% Driving Deaths with Alcohol Involvement':'%_Driving_Deaths_with_Alcohol_Involvement',
         'Sexually transmitted infections_Chlamydia Rate':'Chlamydia_Cases_per_100K',
         'Teen births_Teen Birth Rate':'Teen_Birth_Rate',
         'Uninsured_% Uninsured':'%_Ppl_Under65_w/o_Insurance',
         'Primary care physicians_Primary Care Physicians Rate':'PCP_per_100K',
         'Dentists_Dentist Rate':'Dentists_per_100K',
         'Mental health providers_Mental Health Provider Rate':'MHP_per_100K',
         'Preventable hospital stays_Preventable Hospitalization Rate':'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
         'High school graduation_High School Graduation Rate':'Graduation_Rate',
         'Some college_% Some College':'%_Adults(25-44)_with_Some_Post-secondary_Ed',
         'Unemployment_% Unemployed':'%_Ppl(16+)_Unemployed_&_Looking_for_Work',
         'Children in poverty_% Children in Poverty':'%_Children(under18)_Living_in_Poverty',
         'Income inequality_80th Percentile Income':'80th_Percentile_MHI',
         'Income inequality_20th Percentile Income':'20th_Percentile_MHI',
         'Children in single-parent households_% Single-Parent Households':'%_Children_Living_in_Single_Parent_Household',
         'Social associations_Social Association Rate':'Associations_per_10K',
         'Violent crime_Annual Average Violent Crimes':'Annual_Avg_Violent_Crimes',
         'Violent crime_Violent Crime Rate':'Violent_Crimes_per_100K',
         'Injury deaths_Injury Death Rate':'Injury_Death_Rate_per_100K',
         'Air pollution - particulate matter_Average Daily PM2.5':'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
         'Drinking water violations_Presence of Water Violation':'County_Affected_by_Water_Violation(1=Yes,0=No)',
         'Severe housing problems_% Severe Housing Problems':'%_Households_with_One_or_More Problems(overcrowding,high-housing-cost,lack-kitchen,lack-plumbing-facilities)',
         'Long commute - driving alone_% Long Commute - Drives Alone':'%_Lone_Commuters_who_Commute_More_Than_30Min'}, 
                  inplace = True)
        
        # Drop unnecessary columns
        df = df.drop(['Premature death_95% CI - Low',
         'Premature death_95% CI - High',
         'Premature death_YPLL Rate (AIAN) 95% CI - Low',
         'Premature death_YPLL Rate (AIAN) 95% CI - High',
         'Premature death_YPLL Rate (Asian) 95% CI - Low',
         'Premature death_YPLL Rate (Asian) 95% CI - High',
         'Premature death_YPLL Rate (Black) 95% CI - Low',
         'Premature death_YPLL Rate (Black) 95% CI - High',
         'Premature death_YPLL Rate (Hispanic) 95% CI - Low',
         'Premature death_YPLL Rate (Hispanic) 95% CI - High',
         'Premature death_YPLL Rate (White) 95% CI - Low',
         'Premature death_YPLL Rate (White) 95% CI - High',
         'Premature death_Z-Score',
         'Poor or fair health_95% CI - Low',
         'Poor or fair health_95% CI - High',
         'Poor physical health days_Average Number of Physically Unhealthy Days',
         'Poor mental health days_Average Number of Mentally Unhealthy Days',
         'Low birthweight_Unreliable',
         'Poor physical health days_95% CI - Low',
         'Poor physical health days_95% CI - High',
         'Poor mental health days_95% CI - Low',
         'Poor mental health days_95% CI - High',
         'Low birthweight_95% CI - Low',
         'Low birthweight_95% CI - High',
         'Low birthweight_% LBW (AIAN) 95% CI - Low',
         'Low birthweight_% LBW (AIAN) 95% CI - High',
         'Low birthweight_% LBW (Asian) 95% CI - Low',
         'Low birthweight_% LBW (Asian) 95% CI - High',
         'Low birthweight_% LBW (Black) 95% CI - Low',
         'Low birthweight_% LBW (Black) 95% CI - High',
         'Low birthweight_% LBW (Hispanic) 95% CI - Low',
         'Low birthweight_% LBW (Hispanic) 95% CI - High',
         'Low birthweight_% LBW (White) 95% CI - Low',
         'Low birthweight_% LBW (White) 95% CI - High',
         'Adult smoking_95% CI - Low',
         'Adult smoking_95% CI - High',
         'Adult obesity_95% CI - Low',
         'Adult obesity_95% CI - High',
         'Physical inactivity_95% CI - Low',
         'Physical inactivity_95% CI - High',
         'Excessive drinking_95% CI - Low',
         'Excessive drinking_95% CI - High',
         'Alcohol-impaired driving deaths_95% CI - Low',
         'Alcohol-impaired driving deaths_95% CI - High',
         'Teen births_95% CI - Low',
         'Teen births_95% CI - High',
         'Teen births_Teen Birth Rate (AIAN) 95% CI - Low',
         'Teen births_Teen Birth Rate (AIAN) 95% CI - High',
         'Teen births_Teen Birth Rate (Asian) 95% CI - Low',
         'Teen births_Teen Birth Rate (Asian) 95% CI - High',
         'Teen births_Teen Birth Rate (Black) 95% CI - Low',
         'Teen births_Teen Birth Rate (Black) 95% CI - High',
         'Teen births_Teen Birth Rate (Hispanic) 95% CI - Low',
         'Teen births_Teen Birth Rate (Hispanic) 95% CI - High',
         'Teen births_Teen Birth Rate (White) 95% CI - Low',
         'Teen births_Teen Birth Rate (White) 95% CI - High',
         'Uninsured_95% CI - Low',
         'Uninsured_95% CI - High',
         'Some college_95% CI - Low',
         'Some college_95% CI - High',
         'Children in poverty_95% CI - Low',
         'Children in poverty_95% CI - High',
         'Children in single-parent households_95% CI - Low',
         'Children in single-parent households_95% CI - High',
         'Injury deaths_95% CI - Low',
         'Injury deaths_95% CI - High',
         'Injury deaths_Injury Death Rate (AIAN) 95% CI - Low',
         'Injury deaths_Injury Death Rate (AIAN) 95% CI - High',
         'Injury deaths_Injury Death Rate (Asian) 95% CI - Low',
         'Injury deaths_Injury Death Rate (Asian) 95% CI - High',
         'Injury deaths_Injury Death Rate (Black) 95% CI - Low',
         'Injury deaths_Injury Death Rate (Black) 95% CI - High',
         'Injury deaths_Injury Death Rate (Hispanic) 95% CI - Low',
         'Injury deaths_Injury Death Rate (Hispanic) 95% CI - High',
         'Injury deaths_Injury Death Rate (White) 95% CI - Low',
         'Injury deaths_Injury Death Rate (White) 95% CI - High',
         'Severe housing problems_95% CI - Low',
         'Severe housing problems_95% CI - High',
         'Severe housing problems_Severe Housing Cost Burden 95% CI - Low',
         'Severe housing problems_Severe Housing Cost Burden 95% CI - High',
         'Severe housing problems_Overcrowding 95% CI - Low',
         'Severe housing problems_Overcrowding 95% CI - High',
         'Severe housing problems_Inadequate Facilities 95% CI - Low',
         'Severe housing problems_Inadequate Facilities 95% CI - High',
         'Driving alone to work_95% CI - Low',
         'Driving alone to work_95% CI - High',
         'Driving alone to work_% Drive Alone (AIAN) 95% CI - Low',
         'Driving alone to work_% Drive Alone (AIAN) 95% CI - High',
         'Driving alone to work_% Drive Alone (Asian) 95% CI - Low',
         'Driving alone to work_% Drive Alone (Asian) 95% CI - High',
         'Driving alone to work_% Drive Alone (Black) 95% CI - Low',
         'Driving alone to work_% Drive Alone (Black) 95% CI - High',
         'Driving alone to work_% Drive Alone (Hispanic) 95% CI - Low',
         'Driving alone to work_% Drive Alone (Hispanic) 95% CI - High',
         'Driving alone to work_% Drive Alone (White) 95% CI - Low',
         'Driving alone to work_% Drive Alone (White) 95% CI - High',
         'Long commute - driving alone_95% CI - Low',
         'Long commute - driving alone_95% CI - High',
         'Poor or fair health_Z-Score',
         'Poor physical health days_Z-Score',
         'Poor mental health days_Z-Score',
         'Low birthweight_Z-Score',
         'Adult smoking_Z-Score',
         'Adult obesity_Z-Score',
         'Food environment index_Z-Score',
         'Physical inactivity_Z-Score',
         'Access to exercise opportunities_Z-Score',
         'Excessive drinking_Z-Score',
         'Alcohol-impaired driving deaths_Z-Score',
         'Sexually transmitted infections_Z-Score',
         'Teen births_Z-Score',
         'Uninsured_Z-Score',
         'Primary care physicians_Z-Score',
         'Dentists_Z-Score',
         'Mental health providers_Z-Score',
         'Preventable hospital stays_Z-Score',
         'Mammography screening_Z-Score',
         'Flu vaccinations_Z-Score',
         'High school graduation_Z-Score',
         'Some college_Z-Score',
         'Unemployment_Z-Score',
         'Children in poverty_Z-Score',
         'Income inequality_Z-Score',
         'Children in single-parent households_Z-Score',
         'Social associations_Z-Score',
         'Violent crime_Z-Score',
         'Injury deaths_Z-Score',
         'Drinking water violations_Z-Score',
         'Severe housing problems_Z-Score',
         'Driving alone to work_Z-Score',
         'Long commute - driving alone_Z-Score',
         'Premature death_YPLL Rate (AIAN)',
         'Premature death_YPLL Rate (Asian)',
         'Premature death_YPLL Rate (Black)',
         'Premature death_YPLL Rate (Hispanic)',
         'Premature death_YPLL Rate (White)',
         'Low birthweight_% LBW (AIAN)',
         'Low birthweight_% LBW (Asian)',
         'Low birthweight_% LBW (Black)',
         'Low birthweight_% LBW (Hispanic)',
         'Low birthweight_% LBW (White)',
         'Teen births_Teen Birth Rate (AIAN)',
         'Teen births_Teen Birth Rate (Asian)',
         'Teen births_Teen Birth Rate (Black)',
         'Teen births_Teen Birth Rate (Hispanic)',
         'Teen births_Teen Birth Rate (White)',
         'Preventable hospital stays_Preventable Hosp. Rate (AIAN)',
         'Preventable hospital stays_Preventable Hosp. Rate (Asian)',
         'Preventable hospital stays_Preventable Hosp. Rate (Black)',
         'Preventable hospital stays_Preventable Hosp. Rate (Hispanic)',
         'Preventable hospital stays_Preventable Hosp. Rate (White)',
         'Mammography screening_% Screened (AIAN)',
         'Mammography screening_% Screened (Asian)',
         'Mammography screening_% Screened (Black)',
         'Mammography screening_% Screened (Hispanic)',
         'Mammography screening_% Screened (White)',
         'Flu vaccinations_% Vaccinated (AIAN)',
         'Flu vaccinations_% Vaccinated (Asian)',
         'Flu vaccinations_% Vaccinated (Black)',
         'Flu vaccinations_% Vaccinated (Hispanic)',
         'Flu vaccinations_% Vaccinated (White)',
         'Children in poverty_% Children in Poverty (AIAN)',
         'Children in poverty_% Children in Poverty (Asian)',
         'Children in poverty_% Children in Poverty (Black)',
         'Children in poverty_% Children in Poverty (Hispanic)',
         'Children in poverty_% Children in Poverty (White)',
         'Injury deaths_Injury Death Rate (AIAN)',
         'Injury deaths_Injury Death Rate (Asian)',
         'Injury deaths_Injury Death Rate (Black)',
         'Injury deaths_Injury Death Rate (Hispanic)',
         'Injury deaths_Injury Death Rate (White)',
         'Driving alone to work_% Drive Alone (AIAN)',
         'Driving alone to work_% Drive Alone (Asian)',
         'Driving alone to work_% Drive Alone (Black)',
         'Driving alone to work_% Drive Alone (Hispanic)',
         'Driving alone to work_% Drive Alone (White)',
         'Flu vaccinations_% Vaccinated',
         'Air pollution - particulate matter_Z-Score',
         'Alcohol-impaired driving deaths_# Alcohol-Impaired Driving Deaths',
         'Alcohol-impaired driving deaths_# Driving Deaths',
         'Sexually transmitted infections_# Chlamydia Cases',
         'Uninsured_# Uninsured',
         'Primary care physicians_# Primary Care Physicians',
         'Primary care physicians_Primary Care Physicians Ratio',
         'Dentists_# Dentists',
         'Dentists_Dentist Ratio',
         'Mental health providers_# Mental Health Providers',
         'Mental health providers_Mental Health Provider Ratio',
         'Mammography screening_% With Annual Mammogram',
         'High school graduation_Cohort Size',
         'Some college_# Some College',
         'Some college_Population',
         'Unemployment_# Unemployed',
         'Unemployment_Labor Force',
         'Income inequality_Income Ratio',
         'Children in single-parent households_# Single-Parent Households',
         'Children in single-parent households_# Households',
         'Social associations_# Associations',
         'Injury deaths_# Injury Deaths',
         'Severe housing problems_Severe Housing Cost Burden',
         'Severe housing problems_Overcrowding',
         'Severe housing problems_Inadequate Facilities',
         'Driving alone to work_% Drive Alone to Work',
         'Long commute - driving alone_# Workers who Drive Alone']
                     ,axis=1)
        
        
        # Clean Additional Data Second by addressing multiindex column names
        df2.columns = df2.columns.map('_'.join)
        
        # Rename Columns
        df2.rename(columns = {'Unnamed: 0_level_0_FIPS':'FIPS',
         'Unnamed: 1_level_0_State':'State',
         'Unnamed: 2_level_0_County':'County/Borough/Parish',
         'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
         'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
         'Premature age-adjusted mortality_Age-Adjusted Death Rate':'Premature(under75)_Age_Adjusted_Mortality_per_100K',
         'Child mortality_Child Mortality Rate':'Child(under18)_Mortality_per_100K',
         'Infant mortality_Infant Mortality Rate':'Infant(within1yr)_Mortality_per_1K_live_births',
         'Diabetes prevalence_% Adults with Diabetes':'%_Adults(20+)_Diagnosed_Diabetes',
         'HIV prevalence_HIV Prevalence Rate':'#_Ppl(Age13+)_diagnosed_HIV_per_100K',
         'Food insecurity_% Food Insecure':'%_Pop_w/o_Adequate_Access_to_Food',
         'Drug overdose deaths_Drug Overdose Mortality Rate':'Drug_Overdose_Deaths_per_100K',
         'Motor vehicle crash deaths_Motor Vehicle Mortality Rate':'Motor_Vehicle_Deaths_per_100K',
         'Uninsured adults_% Uninsured':'%_Adults(under65)_w/o_Health_Insurance',
         'Uninsured children_% Uninsured':'%_Children(under19)_w/o_Health_Insurance',
         'Other primary care providers_Other Primary Care Provider Rate':'Other_Primary_Care_Providers(non_physician)_per_100K',
         'Median household income_Median Household Income':'Median_Household_Income',
         'Children eligible for free or reduced price lunch_% Enrolled in Free or Reduced Lunch':'%_Children_Enrolled_in_Free/Reduced_Lunch',
         'Homicides_Homicide Rate':'#_Deaths_due_to_Homicide_per_100K',
         'Demographics_Population':'Resident_Population',
         'Demographics_% less than 18 years of age':'%_under_18_years_of_age',
         'Demographics_% 65 and over':'%_Age_65_and_over',
         'Demographics_# Black':'#_Non-Hispanic_Black_or_African_American',
         'Demographics_% Black':'%_Non-Hispanic_Black_or_African_American',
         'Demographics_# American Indian & Alaska Native':'#_American_Indian_or_Alaskan_Native',
         'Demographics_% American Indian & Alaska Native':'%_American_Indian_or_Alaskan_Native',
         'Demographics_# Asian':'#_Asian',
         'Demographics_% Asian':'%_Asian',
         'Demographics_# Native Hawaiian/Other Pacific Islander':'#_Native_Hawaiian/Other_Pacific_Islander',
         'Demographics_% Native Hawaiian/Other Pacific Islander':'%_Native_Hawaiian/Other_Pacific_Islander',
         'Demographics_# Hispanic':'#_Hispanic',
         'Demographics_% Hispanic':'%_Hispanic',
         'Demographics_# Non-Hispanic White':'#_Non-Hispanic_White',
         'Demographics_% Non-Hispanic White':'%_Non-Hispanic_White',
         'Demographics_# Not Proficient in English':'#_Not_Proficient_in_English',
         'Demographics_% Not Proficient in English':'%_Not_Proficient_in_English',
         'Demographics_% Female':'%_Female',
         'Demographics_# Rural':'#_Pop_Living_in_Rural_Area',
         'Demographics_% Rural':'%_Rural'}, 
                   inplace = True)
        
        # Drop unnecessary columns
        df2 = df2.drop(['Life expectancy_Life Expectancy',
         'Life expectancy_95% CI - Low',
         'Life expectancy_95% CI - High',
         'Life expectancy_Life Expectancy (AIAN)',
         'Life expectancy_Life Expectancy (AIAN) 95% CI - Low',
         'Life expectancy_Life Expectancy (AIAN) 95% CI - High',
         'Life expectancy_Life Expectancy (Asian)',
         'Life expectancy_Life Expectancy (Asian) 95% CI - Low',
         'Life expectancy_Life Expectancy (Asian) 95% CI - High',
         'Life expectancy_Life Expectancy (Black)',
         'Life expectancy_Life Expectancy (Black) 95% CI - Low',
         'Life expectancy_Life Expectancy (Black) 95% CI - High',
         'Life expectancy_Life Expectancy (Hispanic)',
         'Life expectancy_Life Expectancy (Hispanic) 95% CI - Low',
         'Life expectancy_Life Expectancy (Hispanic) 95% CI - High',
         'Life expectancy_Life Expectancy (White)',
         'Life expectancy_Life Expectancy (White) 95% CI - Low',
         'Life expectancy_Life Expectancy (White) 95% CI - High',
         'Premature age-adjusted mortality_95% CI - Low',
         'Premature age-adjusted mortality_95% CI - High',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (AIAN)',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (AIAN) 95% CI - Low',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (AIAN) 95% CI - High',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Asian)',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Asian) 95% CI - Low',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Asian) 95% CI - High',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Black)',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Black) 95% CI - Low',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Black) 95% CI - High',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Hispanic)',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Hispanic) 95% CI - Low',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Hispanic) 95% CI - High',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (White)',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (White) 95% CI - Low',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (White) 95% CI - High',
         'Child mortality_95% CI - Low',
         'Child mortality_95% CI - High',
         'Child mortality_Child Mortality Rate (AIAN)',
         'Child mortality_Child Mortality Rate (AIAN) 95% CI - Low',
         'Child mortality_Child Mortality Rate (AIAN) 95% CI - High',
         'Child mortality_Child Mortality Rate (Asian)',
         'Child mortality_Child Mortality Rate (Asian) 95% CI - Low',
         'Child mortality_Child Mortality Rate (Asian) 95% CI - High',
         'Child mortality_Child Mortality Rate (Black)',
         'Child mortality_Child Mortality Rate (Black) 95% CI - Low',
         'Child mortality_Child Mortality Rate (Black) 95% CI - High',
         'Child mortality_Child Mortality Rate (Hispanic)',
         'Child mortality_Child Mortality Rate (Hispanic) 95% CI - Low',
         'Child mortality_Child Mortality Rate (Hispanic) 95% CI - High',
         'Child mortality_Child Mortality Rate (White)',
         'Child mortality_Child Mortality Rate (White) 95% CI - Low',
         'Child mortality_Child Mortality Rate (White) 95% CI - High',
         'Infant mortality_95% CI - Low',
         'Infant mortality_95% CI - High',
         'Infant mortality_Infant Mortality Rate (AIAN)',
         'Infant mortality_Infant Mortality Rate (AIAN) 95% CI - Low',
         'Infant mortality_Infant Mortality Rate (AIAN) 95% CI - High',
         'Infant mortality_Infant Mortality Rate (Asian)',
         'Infant mortality_Infant Mortality Rate (Asian) 95% CI - Low',
         'Infant mortality_Infant Mortality Rate (Asian) 95% CI - High',
         'Infant mortality_Infant Mortality Rate (Black)',
         'Infant mortality_Infant Mortality Rate (Black) 95% CI - Low',
         'Infant mortality_Infant Mortality Rate (Black) 95% CI - High',
         'Infant mortality_Infant Mortality Rate (Hispanic)',
         'Infant mortality_Infant Mortality Rate (Hispanic) 95% CI - Low',
         'Infant mortality_Infant Mortality Rate (Hispanic) 95% CI - High',
         'Infant mortality_Infant Mortality Rate (White)',
         'Infant mortality_Infant Mortality Rate (White) 95% CI - Low',
         'Infant mortality_Infant Mortality Rate (White) 95% CI - High',
         'Frequent physical distress_% Frequent Physical Distress',
         'Frequent physical distress_95% CI - Low',
         'Frequent physical distress_95% CI - High',
         'Frequent mental distress_% Frequent Mental Distress',
         'Frequent mental distress_95% CI - Low',
         'Frequent mental distress_95% CI - High',
         'Diabetes prevalence_95% CI - Low',
         'Diabetes prevalence_95% CI - High',
         'Drug overdose deaths_95% CI - Low',
         'Drug overdose deaths_95% CI - High',
         'Drug overdose deaths_Drug Overdose Mortality Rate (AIAN)',
         'Drug overdose deaths_Drug Overdose Mortality Rate (AIAN) 95% CI - Low',
         'Drug overdose deaths_Drug Overdose Mortality Rate (AIAN) 95% CI - High',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Asian)',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Asian) 95% CI - Low',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Asian) 95% CI - High',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Black)',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Black) 95% CI - Low',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Black) 95% CI - High',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Hispanic)',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Hispanic) 95% CI - Low',
         'Drug overdose deaths_Drug Overdose Mortality Rate (Hispanic) 95% CI - High',
         'Drug overdose deaths_Drug Overdose Mortality Rate (White)',
         'Drug overdose deaths_Drug Overdose Mortality Rate (White) 95% CI - Low',
         'Drug overdose deaths_Drug Overdose Mortality Rate (White) 95% CI - High',
         'Motor vehicle crash deaths_95% CI - Low',
         'Motor vehicle crash deaths_95% CI - High',
         'Motor vehicle crash deaths_MV Mortality Rate (AIAN)',
         'Motor vehicle crash deaths_MV Mortality Rate (AIAN) 95% CI - Low',
         'Motor vehicle crash deaths_MV Mortality Rate (AIAN) 95% CI - High',
         'Motor vehicle crash deaths_MV Mortality Rate (Asian)',
         'Motor vehicle crash deaths_MV Mortality Rate (Asian) 95% CI - Low',
         'Motor vehicle crash deaths_MV Mortality Rate (Asian) 95% CI - High',
         'Motor vehicle crash deaths_MV Mortality Rate (Black)',
         'Motor vehicle crash deaths_MV Mortality Rate (Black) 95% CI - Low',
         'Motor vehicle crash deaths_MV Mortality Rate (Black) 95% CI - High',
         'Motor vehicle crash deaths_MV Mortality Rate (Hispanic)',
         'Motor vehicle crash deaths_MV Mortality Rate (Hispanic) 95% CI - Low',
         'Motor vehicle crash deaths_MV Mortality Rate (Hispanic) 95% CI - High',
         'Motor vehicle crash deaths_MV Mortality Rate (White)',
         'Motor vehicle crash deaths_MV Mortality Rate (White) 95% CI - Low',
         'Motor vehicle crash deaths_MV Mortality Rate (White) 95% CI - High',
         'Insufficient sleep_% Insufficient Sleep',
         'Insufficient sleep_95% CI - Low',
         'Insufficient sleep_95% CI - High',
         'Uninsured adults_95% CI - Low',
         'Uninsured adults_95% CI - High',
         'Uninsured children_95% CI - Low',
         'Uninsured children_95% CI - High',
         'Disconnected youth_% Disconnected Youth',
         'Disconnected youth_95% CI - Low',
         'Disconnected youth_95% CI - High',
         'Reading scores_Average Grade Performance',
         'Reading scores_Average Grade Performance (Asian)',
         'Reading scores_Average Grade Performance (Black)',
         'Reading scores_Average Grade Performance (Hispanic)',
         'Reading scores_Average Grade Performance (White)',
         'Math scores_Average Grade Performance',
         'Math scores_Average Grade Performance (Asian)',
         'Math scores_Average Grade Performance (Black)',
         'Math scores_Average Grade Performance (Hispanic)',
         'Math scores_Average Grade Performance (White)',
         'Median household income_95% CI - Low',
         'Median household income_95% CI - High',
         'Median household income_Household Income (AIAN)',
         'Median household income_Household Income (AIAN) 95% CI - Low',
         'Median household income_Household Income (AIAN) 95% CI - High',
         'Median household income_Household Income (Asian)',
         'Median household income_Household Income (Asian) 95% CI - Low',
         'Median household income_Household Income (Asian) 95% CI - High',
         'Median household income_Household Income (Black)',
         'Median household income_Household Income (Black) 95% CI - Low',
         'Median household income_Household Income (Black) 95% CI - High',
         'Median household income_Household Income (Hispanic)',
         'Median household income_Household Income (Hispanic) 95% CI - Low',
         'Median household income_Household Income (Hispanic) 95% CI - High',
         'Median household income_Household Income (White)',
         'Median household income_Household Income (White) 95% CI - Low',
         'Median household income_Household Income (White) 95% CI - High',
         'Residential segregation - Black/White_Segregation index',
         'Residential segregation - non-White/White_Segregation Index',
         'Homicides_95% CI - Low',
         'Homicides_95% CI - High',
         'Homicides_Homicide Rate (AIAN)',
         'Homicides_Homicide Rate (AIAN) 95% CI - Low',
         'Homicides_Homicide Rate (AIAN) 95% CI - High',
         'Homicides_Homicide Rate (Asian)',
         'Homicides_Homicide Rate (Asian) 95% CI - Low',
         'Homicides_Homicide Rate (Asian) 95% CI - High',
         'Homicides_Homicide Rate (Black)',
         'Homicides_Homicide Rate (Black) 95% CI - Low',
         'Homicides_Homicide Rate (Black) 95% CI - High',
         'Homicides_Homicide Rate (Hispanic)',
         'Homicides_Homicide Rate (Hispanic) 95% CI - Low',
         'Homicides_Homicide Rate (Hispanic) 95% CI - High',
         'Homicides_Homicide Rate (White)',
         'Homicides_Homicide Rate (White) 95% CI - Low',
         'Homicides_Homicide Rate (White) 95% CI - High',
         'Suicides_# Deaths',
         'Suicides_Suicide Rate (Age-Adjusted)',
         'Suicides_95% CI - Low',
         'Suicides_95% CI - High',
         'Suicides_Crude Rate',
         'Suicides_Suicide Rate (AIAN)',
         'Suicides_Suicide Rate (AIAN) 95% CI - Low',
         'Suicides_Suicide Rate (AIAN) 95% CI - High',
         'Suicides_Suicide Rate (Asian)',
         'Suicides_Suicide Rate (Asian) 95% CI - Low',
         'Suicides_Suicide Rate (Asian) 95% CI - High',
         'Suicides_Suicide Rate (Black)',
         'Suicides_Suicide Rate (Black) 95% CI - Low',
         'Suicides_Suicide Rate (Black) 95% CI - High',
         'Suicides_Suicide Rate (Hispanic)',
         'Suicides_Suicide Rate (Hispanic) 95% CI - Low',
         'Suicides_Suicide Rate (Hispanic) 95% CI - High',
         'Suicides_Suicide Rate (White)',
         'Suicides_Suicide Rate (White) 95% CI - Low',
         'Suicides_Suicide Rate (White) 95% CI - High',
         'Firearm fatalities_# Firearm Fatalities',
         'Firearm fatalities_Firearm Fatalities Rate',
         'Firearm fatalities_95% CI - Low',
         'Firearm fatalities_95% CI - High',
         'Firearm fatalities_Firearm Fatalities Rate (AIAN)',
         'Firearm fatalities_Firearm Fatalities Rate (AIAN) 95% CI - Low',
         'Firearm fatalities_Firearm Fatalities Rate (AIAN) 95% CI - High',
         'Firearm fatalities_Firearm Fatalities Rate (Asian)',
         'Firearm fatalities_Firearm Fatalities Rate (Asian) 95% CI - Low',
         'Firearm fatalities_Firearm Fatalities Rate (Asian) 95% CI - High',
         'Firearm fatalities_Firearm Fatalities Rate (Black)',
         'Firearm fatalities_Firearm Fatalities Rate (Black) 95% CI - Low',
         'Firearm fatalities_Firearm Fatalities Rate (Black) 95% CI - High',
         'Firearm fatalities_Firearm Fatalities Rate (Hispanic)',
         'Firearm fatalities_Firearm Fatalities Rate (Hispanic) 95% CI - Low',
         'Firearm fatalities_Firearm Fatalities Rate (Hispanic) 95% CI - High',
         'Firearm fatalities_Firearm Fatalities Rate (White)',
         'Firearm fatalities_Firearm Fatalities Rate (White) 95% CI - Low',
         'Firearm fatalities_Firearm Fatalities Rate (White) 95% CI - High',
         'Juvenile arrests_Non-Petitioned Cases',
         'Juvenile arrests_Petitioned Cases',
         'Juvenile arrests_Denominator',
         'Juvenile arrests_Juvenile Arrest Rate',
         'Traffic volume_Average Traffic Volume per Meter of Major Roadways',
         'Homeownership_# Homeowners',
         'Homeownership_% Homeowners',
         'Homeownership_95% CI - Low',
         'Homeownership_95% CI - High',
         'Severe housing cost burden_# Households with Severe Cost Burden',
         'Severe housing cost burden_% Severe Housing Cost Burden',
         'Severe housing cost burden_95% CI - Low',
         'Severe housing cost burden_95% CI - High',
         'Demographics_95% CI - Low',
         'Demographics_95% CI - High',
         'Premature age-adjusted mortality_# Deaths',
         'Child mortality_# Deaths',
         'Infant mortality_# Deaths',
         'HIV prevalence_# HIV Cases',
         'Food insecurity_# Food Insecure',
         'Limited access to healthy foods_# Limited Access',
         'Limited access to healthy foods_% Limited Access to Healthy Foods',
         'Drug overdose deaths_# Drug Overdose Deaths',
         'Motor vehicle crash deaths_# Motor Vehicle Deaths',
         'Uninsured adults_# Uninsured',
         'Uninsured children_# Uninsured',
         'Other primary care providers_Other Primary Care Provider Ratio']
                       ,axis=1)
        
        # Join two dataframes
        joined_df = pd.merge(df, df2, on='FIPS', how='outer')
        joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
        joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        joined_df['Year'] = '2020'
        
        # append to list
        appended_data2020.append(joined_df)
        
#convert to dataframe
appended_data2020 = pd.concat(appended_data2020)

2020 County Health Rankings Alabama Data - v1_1.xlsx
2020 County Health Rankings Alaska Data - v1_0.xlsx
2020 County Health Rankings Arizona Data - v1_0.xlsx
2020 County Health Rankings Arkansas Data - v1_0.xlsx
2020 County Health Rankings California Data - v1_0.xlsx
2020 County Health Rankings Colorado Data - v1_0.xlsx
2020 County Health Rankings Connecticut Data - v1_0.xlsx
2020 County Health Rankings Delaware Data - v1.xlsx
2020 County Health Rankings District of Columbia Data - v1_0.xlsx
2020 County Health Rankings Florida Data - v1_0.xlsx
2020 County Health Rankings Georgia Data - v1_0.xlsx
2020 County Health Rankings Hawaii Data - v1_0.xlsx
2020 County Health Rankings Idaho Data - v1_0.xlsx
2020 County Health Rankings Illinois Data - v1_0.xlsx
2020 County Health Rankings Indiana Data - v1_0.xlsx
2020 County Health Rankings Iowa Data - v1_0.xlsx
2020 County Health Rankings Kansas Data - v1_0.xlsx


In [3]:
appended_data2020.head(20)

,FIPS,State,County/Borough/Parish,Premature_Deaths,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,Avg_Days_Poor_Physical_Health_per_Month,Avg_Days_Mentally_Unhealthy_per_Month,Low_BirthWt_Unreliable,%_Low_Birthweight_Births,...,#_Hispanic,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Year
0,1000,Alabama,NaN,81791.0,9942.794666,22.028703,4.918052,4.939753,NaN,10.254871,...,217181,4.443264,3197324,65.413428,48517,1.061048,51.633032,1957932.0,40.963183,2020
1,1001,Alabama,Autauga,791.0,8128.591190,20.882987,4.743889,4.654031,NaN,8.619529,...,1649,2.965774,41316,74.308016,426,0.820225,51.448715,22921.0,42.002162,2020
2,1003,Alabama,Baldwin,2967.0,7354.122530,17.509134,4.224519,4.304056,NaN,8.345003,...,10131,4.646779,181201,83.111337,1068,0.543517,51.538377,77060.0,42.279099,2020
3,1005,Alabama,Barbour,472.0,10253.573403,29.591802,5.430279,5.185594,NaN,11.474559,...,1064,4.276355,11356,45.641252,398,1.631683,47.216752,18613.0,67.789635,2020
4,1007,Alabama,Bibb,471.0,11977.539484,19.439724,4.591482,4.551699,NaN,10.308710,...,588,2.625000,16708,74.589286,57,0.268210,46.781250,15663.0,68.352607,2020
5,1009,Alabama,Blount,1085.0,11335.071134,21.745293,4.860808,4.891455,NaN,7.604563,...,5536,9.571231,50255,86.886238,934,1.724520,50.726141,51562.0,89.951502,2020
6,1011,Alabama,Bullock,203.0,11679.558981,30.999102,5.359159,4.891351,NaN,15.688488,...,807,7.960150,2151,21.217203,43,0.440754,45.482344,5607.0,51.374382,2020
7,1013,Alabama,Butler,423.0,14359.939103,27.910673,5.400806,5.340750,NaN,12.680288,...,297,1.509146,10087,51.255081,93,0.494155,53.429878,14921.0,71.232157,2020
8,1015,Alabama,Calhoun,2373.0,12078.616145,23.107733,4.880377,4.816072,NaN,9.161374,...,4469,3.910673,82308,72.024992,1076,0.991376,51.946586,39955.0,33.696826,2020
9,1017,Alabama,Chambers,654.0,11112.536614,23.995474,4.941178,4.873581,NaN,12.213740,...,861,2.561357,18566,55.231296,36,0.112938,52.125539,16816.0,49.148034,2020


In [4]:
list(appended_data2020.columns.values)

['FIPS',
 'State',
 'County/Borough/Parish',
 'Premature_Deaths',
 'Yrs_Potential_Life_Lost_per100K(YPLL)',
 '%_Fair/Poor_Health',
 'Avg_Days_Poor_Physical_Health_per_Month',
 'Avg_Days_Mentally_Unhealthy_per_Month',
 'Low_BirthWt_Unreliable',
 '%_Low_Birthweight_Births',
 '%_Adult_Smokers',
 '%_Adult_Obesity',
 'Healthy_Food_Access(0=Worst,10=Best)',
 '%_Adults_Physical_Inactivity',
 '%_Access_to_Excercise_Opportunities',
 '%_Adults_Excessive_Drinking',
 '#_Alcohol-Impaired_Motor_Vehicle_Deaths',
 '#_Motor_Vehicle_Deaths_x',
 '%_Driving_Deaths_with_Alcohol_Involvement',
 '#_Chlamydia_Cases',
 'Chlamydia_Cases_per_100K',
 'Teen_Birth_Rate',
 '#_Ppl_Under65_w/o_Insurance',
 '%_Ppl_Under65_w/o_Insurance',
 '#_Primary_Care_Physicians(PCP)_in_Pt_Care',
 'PCP_per_100K',
 'Population_to_PCP_ratio',
 '#_Dentists',
 'Dentists_per_100K',
 'Population_to_Dentists_ratio',
 '#_Mental_Health_Providers(MHP)',
 'MHP_per_100K',
 'Population_to_MHP_ratio',
 'Preventable_Hospitalization_Rate(per100K_Med

In [5]:
appended_data2020.count()

FIPS                                                    3175
State                                                   3175
County/Borough/Parish                                   3125
Premature_Deaths                                        2885
Yrs_Potential_Life_Lost_per100K(YPLL)                   2885
%_Fair/Poor_Health                                      3175
Avg_Days_Poor_Physical_Health_per_Month                 3175
Avg_Days_Mentally_Unhealthy_per_Month                   3175
Low_BirthWt_Unreliable                                   174
%_Low_Birthweight_Births                                3070
%_Adult_Smokers                                         3175
%_Adult_Obesity                                         3175
Healthy_Food_Access(0=Worst,10=Best)                    3156
%_Adults_Physical_Inactivity                            3175
%_Access_to_Excercise_Opportunities                     3169
%_Adults_Excessive_Drinking                             3175
#_Alcohol-Impaired_Motor

In [6]:
appended_data2020.isnull().any()

FIPS                                                    False
State                                                   False
County/Borough/Parish                                    True
Premature_Deaths                                         True
Yrs_Potential_Life_Lost_per100K(YPLL)                    True
%_Fair/Poor_Health                                      False
Avg_Days_Poor_Physical_Health_per_Month                 False
Avg_Days_Mentally_Unhealthy_per_Month                   False
Low_BirthWt_Unreliable                                   True
%_Low_Birthweight_Births                                 True
%_Adult_Smokers                                         False
%_Adult_Obesity                                         False
Healthy_Food_Access(0=Worst,10=Best)                     True
%_Adults_Physical_Inactivity                            False
%_Access_to_Excercise_Opportunities                      True
%_Adults_Excessive_Drinking                             False
#_Alcoho

In [8]:
appended_data2020.loc[appended_data2020.isnull().any(axis=1)]

,FIPS,State,County/Borough/Parish,Premature_Deaths,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,Avg_Days_Poor_Physical_Health_per_Month,Avg_Days_Mentally_Unhealthy_per_Month,Low_BirthWt_Unreliable,%_Low_Birthweight_Births,...,#_Hispanic,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Year
0,1000,Alabama,NaN,81791.0,9942.794666,22.028703,4.918052,4.939753,NaN,10.254871,...,217181,4.443264,3197324,65.413428,48517,1.061048,51.633032,1957932.0,40.963183,2020
1,1001,Alabama,Autauga,791.0,8128.591190,20.882987,4.743889,4.654031,NaN,8.619529,...,1649,2.965774,41316,74.308016,426,0.820225,51.448715,22921.0,42.002162,2020
2,1003,Alabama,Baldwin,2967.0,7354.122530,17.509134,4.224519,4.304056,NaN,8.345003,...,10131,4.646779,181201,83.111337,1068,0.543517,51.538377,77060.0,42.279099,2020
3,1005,Alabama,Barbour,472.0,10253.573403,29.591802,5.430279,5.185594,NaN,11.474559,...,1064,4.276355,11356,45.641252,398,1.631683,47.216752,18613.0,67.789635,2020
4,1007,Alabama,Bibb,471.0,11977.539484,19.439724,4.591482,4.551699,NaN,10.308710,...,588,2.625000,16708,74.589286,57,0.268210,46.781250,15663.0,68.352607,2020
5,1009,Alabama,Blount,1085.0,11335.071134,21.745293,4.860808,4.891455,NaN,7.604563,...,5536,9.571231,50255,86.886238,934,1.724520,50.726141,51562.0,89.951502,2020
6,1011,Alabama,Bullock,203.0,11679.558981,30.999102,5.359159,4.891351,NaN,15.688488,...,807,7.960150,2151,21.217203,43,0.440754,45.482344,5607.0,51.374382,2020
7,1013,Alabama,Butler,423.0,14359.939103,27.910673,5.400806,5.340750,NaN,12.680288,...,297,1.509146,10087,51.255081,93,0.494155,53.429878,14921.0,71.232157,2020
8,1015,Alabama,Calhoun,2373.0,12078.616145,23.107733,4.880377,4.816072,NaN,9.161374,...,4469,3.910673,82308,72.024992,1076,0.991376,51.946586,39955.0,33.696826,2020
9,1017,Alabama,Chambers,654.0,11112.536614,23.995474,4.941178,4.873581,NaN,12.213740,...,861,2.561357,18566,55.231296,36,0.112938,52.125539,16816.0,49.148034,2020


In [9]:
appended_data2020 = appended_data2020.fillna(value=0)
appended_data2020.head(20)

,FIPS,State,County/Borough/Parish,Premature_Deaths,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,Avg_Days_Poor_Physical_Health_per_Month,Avg_Days_Mentally_Unhealthy_per_Month,Low_BirthWt_Unreliable,%_Low_Birthweight_Births,...,#_Hispanic,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Year
0,1000,Alabama,0,81791.0,9942.794666,22.028703,4.918052,4.939753,0,10.254871,...,217181,4.443264,3197324,65.413428,48517,1.061048,51.633032,1957932.0,40.963183,2020
1,1001,Alabama,Autauga,791.0,8128.591190,20.882987,4.743889,4.654031,0,8.619529,...,1649,2.965774,41316,74.308016,426,0.820225,51.448715,22921.0,42.002162,2020
2,1003,Alabama,Baldwin,2967.0,7354.122530,17.509134,4.224519,4.304056,0,8.345003,...,10131,4.646779,181201,83.111337,1068,0.543517,51.538377,77060.0,42.279099,2020
3,1005,Alabama,Barbour,472.0,10253.573403,29.591802,5.430279,5.185594,0,11.474559,...,1064,4.276355,11356,45.641252,398,1.631683,47.216752,18613.0,67.789635,2020
4,1007,Alabama,Bibb,471.0,11977.539484,19.439724,4.591482,4.551699,0,10.308710,...,588,2.625000,16708,74.589286,57,0.268210,46.781250,15663.0,68.352607,2020
5,1009,Alabama,Blount,1085.0,11335.071134,21.745293,4.860808,4.891455,0,7.604563,...,5536,9.571231,50255,86.886238,934,1.724520,50.726141,51562.0,89.951502,2020
6,1011,Alabama,Bullock,203.0,11679.558981,30.999102,5.359159,4.891351,0,15.688488,...,807,7.960150,2151,21.217203,43,0.440754,45.482344,5607.0,51.374382,2020
7,1013,Alabama,Butler,423.0,14359.939103,27.910673,5.400806,5.340750,0,12.680288,...,297,1.509146,10087,51.255081,93,0.494155,53.429878,14921.0,71.232157,2020
8,1015,Alabama,Calhoun,2373.0,12078.616145,23.107733,4.880377,4.816072,0,9.161374,...,4469,3.910673,82308,72.024992,1076,0.991376,51.946586,39955.0,33.696826,2020
9,1017,Alabama,Chambers,654.0,11112.536614,23.995474,4.941178,4.873581,0,12.213740,...,861,2.561357,18566,55.231296,36,0.112938,52.125539,16816.0,49.148034,2020


In [10]:
appended_data2020.to_csv(os.path.join("Cleaned_Files","2020_County_Health_Ranking_Data_Test.csv"),index=False)

In [11]:
appended_data2019 = []
path = os.path.join("Original_Files","CountyHealthRankingsData","2019")
for WorkingFile in os.listdir(path):
        #identifies the list of files to be grouped together from designated folder
        print(WorkingFile)
        
        # Import the excel file by joining the path directory and file name together and call Ranked Data and Additonal Measure tabs
        file = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Ranked Measure Data',header=(0,1))
        file2 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Additional Measure Data',header=(0,1))
       
        # Load the excel files as two dataframes
        df = pd.DataFrame(file)
        df2 = pd.DataFrame(file2)
        
        # Code from Data_cleaning
       
        # Clean Ranked Data First by addressing multiindex column names
        df.columns = df.columns.map('_'.join)
        
        # Rename Columns
        df.rename(columns = {'Unnamed: 0_level_0_FIPS' : 'FIPS',
         'Unnamed: 1_level_0_State' :'State',
         'Unnamed: 2_level_0_County':'County/Borough/Parish',
         'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
         'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
         'Premature death_Deaths':'Premature_Deaths',
         'Premature death_Years of Potential Life Lost Rate':'Yrs_Potential_Life_Lost_per100K(YPLL)',
         'Poor or fair health_% Fair/Poor':'%_Fair/Poor_Health',
         'Poor physical health days_Physically Unhealthy Days':'Avg_Days_Poor_Physical_Health_per_Month',
         'Poor mental health days_Mentally Unhealthy Days':'Avg_Days_Mentally_Unhealthy_per_Month',
         'Low birthweight_Unreliable':'Low_BirthWt_Unreliable',
         'Low birthweight_% LBW':'%_Low_Birthweight_Births',
         'Adult smoking_% Smokers':'%_Adult_Smokers',
         'Adult obesity_% Obese':'%_Adult_Obesity',
         'Food environment index_Food Environment Index':'Healthy_Food_Access(0=Worst,10=Best)',
         'Physical inactivity_% Physically Inactive':'%_Adults_Physical_Inactivity',
         'Access to exercise opportunities_% With Access':'%_Access_to_Excercise_Opportunities',
         'Excessive drinking_% Excessive Drinking':'%_Adults_Excessive_Drinking',
         'Alcohol-impaired driving deaths_# Alcohol-Impaired Driving Deaths':'#_Alcohol-Impaired_Motor_Vehicle_Deaths',
         'Alcohol-impaired driving deaths_# Driving Deaths':'#_Motor_Vehicle_Deaths',
         'Alcohol-impaired driving deaths_% Alcohol-Impaired':'%_Driving_Deaths_with_Alcohol_Involvement',
         'Sexually transmitted infections_# Chlamydia Cases':'#_Chlamydia_Cases',
         'Sexually transmitted infections_Chlamydia Rate':'Chlamydia_Cases_per_100K',
         'Teen births_Teen Birth Rate':'Teen_Birth_Rate',
         'Uninsured_# Uninsured':'#_Ppl_Under65_w/o_Insurance',
         'Uninsured_% Uninsured':'%_Ppl_Under65_w/o_Insurance',                            
         'Primary care physicians_# Primary Care Physicians':'#_Primary_Care_Physicians(PCP)_in_Pt_Care',
         'Primary care physicians_PCP Rate':'PCP_per_100K',
         'Primary care physicians_PCP Ratio':'Population_to_PCP_ratio',                           
         'Dentists_# Dentists':'#_Dentists',
         'Dentists_Dentist Rate':'Dentists_per_100K',
         'Dentists_Dentist Ratio':'Population_to_Dentists_ratio',
         'Mental health providers_# Mental Health Providers':'#_Mental_Health_Providers(MHP)',
         'Mental health providers_MHP Rate':'MHP_per_100K',
         'Mental health providers_MHP Ratio':'Population_to_MHP_ratio',
         'Preventable hospital stays_Preventable Hosp. Rate':'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
         'Mammography screening_% Screened':'%_Female_Medicare_Enrollees(65-74)_with_Annual_Mammogram',
         'High school graduation_Cohort Size':'#_Students_Expected_to_Graduate',
         'High school graduation_Graduation Rate':'Graduation_Rate',
         'Some college_# Some College':'#_Adults(25-44)_with_Some_Post-secondary_Ed',
         'Some college_Population':'Adults(25-44)',
         'Some college_% Some College':'%_Adults(25-44)_with_Some_Post-secondary_Ed',
         'Unemployment_# Unemployed':'#_Ppl(16+)_Unemployed_&_Looking_for_Work',
         'Unemployment_Labor Force':'Size_of_Labor_Force',
         'Unemployment_% Unemployed':'%_Ppl(16+)_Unemployed_&_Looking_for_Work',
         'Children in poverty_% Children in Poverty':'%_Children(under18)_Living_in_Poverty',
         'Income inequality_80th Percentile Income':'80th_Percentile_MHI',
         'Income inequality_20th Percentile Income':'20th_Percentile_MHI',
         'Income inequality_Income Ratio':'Ratio_of_Household_Income_80th_to_20th',
         'Children in single-parent households_# Single-Parent Households':'#_Children_Living_in_Single_Parent_Household',
         'Children in single-parent households_# Households':'#_Children_in_Households',
         'Children in single-parent households_% Single-Parent Households':'%_Children_Living_in_Single_Parent_Household',
         'Social associations_# Associations':'#_Associations',
         'Social associations_Association Rate':'Associations_per_10K',
         'Violent crime_Annual Average Violent Crimes':'Annual_Avg_Violent_Crimes',
         'Violent crime_Violent Crime Rate':'Violent_Crimes_per_100K',
         'Injury deaths_# Injury Deaths':'#_Injury_Deaths',
         'Injury deaths_Injury Death Rate':'Injury_Death_Rate_per_100K',
         'Air pollution - particulate matter_Average Daily PM2.5':'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
         'Drinking water violations_Presence of violation':'County_Affected_by_Water_Violation(1=Yes,0=No)',
         'Severe housing problems_% Severe Housing Problems':'%_Households_with_One_or_More Problems(overcrowding,high-housing-cost,lack-kitchen,lack-plumbing-facilities)',
         'Severe housing problems_Severe Housing Cost Burden':'%_Households_with_High_Housing_Costs',
         'Severe housing problems_Overcrowding':'%_Households_with_Overcrowding',
         'Severe housing problems_Inadequate Facilities':'%_Households_Lacking_Kitchen_or_Plumbing_Facilities',
         'Driving alone to work_% Drive Alone':'%_Workers_Who_Drive_Alone_to_Work',
         'Long commute - driving alone_# Workers who Drive Alone':'#_Workers_who_Commute(car,truck,van)_Alone',
         'Long commute - driving alone_% Long Commute - Drives Alone':'%_Lone_Commuters_who_Commute_More_Than_30Min'}, 
                  inplace = True)
        
        # Drop unnecessary columns
        df = df.drop([
         'Premature death_95% CI - Low',
         'Premature death_95% CI - High',
         'Premature death_Z-Score',
         'Premature death_YPLL Rate (Black)',
         'Premature death_YPLL Rate (Hispanic)',
         'Premature death_YPLL Rate (White)',
         'Poor or fair health_95% CI - Low',
         'Poor or fair health_95% CI - High',
         'Poor or fair health_Z-Score',
         'Poor physical health days_95% CI - Low',
         'Poor physical health days_95% CI - High',
         'Poor physical health days_Z-Score',
         'Poor mental health days_95% CI - Low',
         'Poor mental health days_95% CI - High',
         'Poor mental health days_Z-Score',
         'Low birthweight_95% CI - Low',
         'Low birthweight_95% CI - High',
         'Low birthweight_Z-Score',
         'Low birthweight_% LBW (Black)',
         'Low birthweight_% LBW (Hispanic)',
         'Low birthweight_% LBW (White)',
         'Adult smoking_95% CI - Low',
         'Adult smoking_95% CI - High',
         'Adult smoking_Z-Score',
         'Adult obesity_95% CI - Low',
         'Adult obesity_95% CI - High',
         'Adult obesity_Z-Score',
         'Food environment index_Z-Score',
         'Physical inactivity_95% CI - Low',
         'Physical inactivity_95% CI - High',
         'Physical inactivity_Z-Score',
         'Access to exercise opportunities_Z-Score',
         'Excessive drinking_95% CI - Low',
         'Excessive drinking_95% CI - High',
         'Excessive drinking_Z-Score',
         'Alcohol-impaired driving deaths_95% CI - Low',
         'Alcohol-impaired driving deaths_95% CI - High',
         'Alcohol-impaired driving deaths_Z-Score',
         'Sexually transmitted infections_Z-Score',
         'Teen births_95% CI - Low',
         'Teen births_95% CI - High',
         'Teen births_Z-Score',
         'Teen births_Teen Birth Rate (Black)',
         'Teen births_Teen Birth Rate (Hispanic)',
         'Teen births_Teen Birth Rate (White)',
         'Uninsured_95% CI - Low',
         'Uninsured_95% CI - High',
         'Uninsured_Z-Score',
         'Primary care physicians_Z-Score',
         'Dentists_Z-Score',
         'Mental health providers_Z-Score',
         'Preventable hospital stays_Z-Score',
         'Preventable hospital stays_Preventable Hosp. Rate (Black)',
         'Preventable hospital stays_Preventable Hosp. Rate (Hispanic)',
         'Preventable hospital stays_Preventable Hosp. Rate (White)',
         'Mammography screening_Z-Score',
         'Mammography screening_% Screened (Black)',
         'Mammography screening_% Screened (Hispanic)',
         'Mammography screening_% Screened (White)',
         'Flu vaccinations_% Vaccinated',
         'Flu vaccinations_Z-Score',
         'Flu vaccinations_% Vaccinated (Black)',
         'Flu vaccinations_% Vaccinated (Hispanic)',
         'Flu vaccinations_% Vaccinated (White)',
         'High school graduation_Z-Score',
         'Some college_95% CI - Low',
         'Some college_95% CI - High',
         'Some college_Z-Score',
         'Unemployment_Z-Score',
         'Children in poverty_95% CI - Low',
         'Children in poverty_95% CI - High',
         'Children in poverty_Z-Score',
         'Children in poverty_% Children in Poverty (Black)',
         'Children in poverty_% Children in Poverty (Hispanic)',
         'Children in poverty_% Children in Poverty (White)',
         'Income inequality_Z-Score',
         'Children in single-parent households_95% CI - Low',
         'Children in single-parent households_95% CI - High',
         'Children in single-parent households_Z-Score',
         'Social associations_Z-Score',
         'Violent crime_Z-Score',
         'Injury deaths_95% CI - Low',
         'Injury deaths_95% CI - High',
         'Injury deaths_Z-Score',
         'Air pollution - particulate matter_Z-Score',
         'Drinking water violations_Z-Score',
         'Severe housing problems_95% CI - Low',
         'Severe housing problems_95% CI - High',
         'Severe housing problems_Z-Score',
         'Driving alone to work_95% CI - Low',
         'Driving alone to work_95% CI - High',
         'Driving alone to work_Z-Score',
         'Driving alone to work_% Drive Alone (Black)',
         'Driving alone to work_% Drive Alone (Hispanic)',
         'Driving alone to work_% Drive Alone (White)',
         'Long commute - driving alone_95% CI - Low',
         'Long commute - driving alone_95% CI - High',
         'Long commute - driving alone_Z-Score']
                             ,axis=1)
        
        # Clean Additional Data Second by addressing multiindex column names
        df2.columns = df2.columns.map('_'.join)
        
        # Rename Columns
        df2.rename(columns = {'Unnamed: 0_level_0_FIPS':'FIPS',
         'Unnamed: 1_level_0_State':'State',
         'Unnamed: 2_level_0_County':'County/Borough/Parish',
         'Unnamed: 2_level_0_Borough':'County/Borough/Parish',
         'Unnamed: 2_level_0_Parish':'County/Borough/Parish',
         'Premature age-adjusted mortality_# Deaths':'Premature(under75)_Age_Adjusted_Mortality_#_Deaths',
         'Premature age-adjusted mortality_Age-Adjusted Death Rate':'Premature(under75)_Age_Adjusted_Mortality_per_100K',
         'Child mortality_# Deaths':'Child(under18)_Mortality_#_Deaths',
         'Child mortality_Child Mortality Rate':'Child(under18)_Mortality_per_100K',
         'Infant mortality_# Deaths':'Infant(within1yr)_Mortality_#_Deaths',
         'Infant mortality_Infant Mortality Rate':'Infant(within1yr)_Mortality_per_1K_live_births',
         'Diabetes prevalence_% Diabetic':'%_Adults(20+)_Diagnosed_Diabetes',
         'HIV prevalence_# HIV Cases':'#_HIV_Cases',
         'HIV prevalence_HIV Prevalence Rate':'#_Ppl(Age13+)_diagnosed_HIV_per_100K',
         'Food insecurity_# Food Insecure':'#_Ppl_Food_Insecure',
         'Food insecurity_% Food Insecure':'%_Pop_w/o_Adequate_Access_to_Food',
         'Limited access to healthy foods_# Limited Access':'#_Ppl_Low_Income_or_Do_Not_Live_Near_Grocery_Store',
         'Limited access to healthy foods_% Limited Access':'%_Pop_with_Limited_Access_to_Food',
         'Drug overdose deaths_# Drug Overdose Deaths':'#_Drug_Overdose_Deaths',
         'Drug overdose deaths_Drug Overdose Mortality Rate':'Drug_Overdose_Deaths_per_100K',
         'Motor vehicle crash deaths_# Motor Vehicle Deaths':'#_Motor_Vehicle_Deaths',
         'Motor vehicle crash deaths_MV Mortality Rate':'Motor_Vehicle_Deaths_per_100K',
         'Uninsured adults_# Uninsured':'#_Adults(under65)_w/o_Health_Insurance',
         'Uninsured adults_% Uninsured':'%_Adults(under65)_w/o_Health_Insurance',
         'Uninsured children_# Uninsured':'#_Children(under19)_w/o_Health_Insurance',
         'Uninsured children_% Uninsured':'%_Children(under19)_w/o_Health_Insurance',
         'Other primary care providers_Other PCP Rate':'Other_Primary_Care_Providers(non_physician)_per_100K',
         'Other primary care providers_Other PCP Ratio':'Pop_to_Other_Primary_Care_Providers(non_physician)',
         'Median household income_Household Income':'Median_Household_Income',
         'Children eligible for free or reduced price lunch_% Free or Reduced Lunch':'%_Children_Enrolled_in_Free/Reduced_Lunch',
         'Homicides_Homicide Rate':'#_Deaths_due_to_Homicide_per_100K',
         'Demographics_Population':'Resident_Population',
         'Demographics_% less than 18 years of age':'%_under_18_years_of_age',
         'Demographics_% 65 and over':'%_Age_65_and_over',
         'Demographics_# Black':'#_Non-Hispanic_Black_or_African_American',
         'Demographics_% Black':'%_Non-Hispanic_Black_or_African_American',
         'Demographics_# American Indian & Alaska Native':'#_American_Indian_or_Alaskan_Native',
         'Demographics_% American Indian & Alaska Native':'%_American_Indian_or_Alaskan_Native',
         'Demographics_# Asian':'#_Asian',
         'Demographics_% Asian':'%_Asian',
         'Demographics_# Native Hawaiian/Other Pacific Islander':'#_Native_Hawaiian/Other_Pacific_Islander',
         'Demographics_% Native Hawaiian/Other Pacific Islander':'%_Native_Hawaiian/Other_Pacific_Islander',
         'Demographics_# Hispanic':'#_Hispanic',
         'Demographics_% Hispanic':'%_Hispanic',
         'Demographics_# Non-Hispanic White':'#_Non-Hispanic_White',
         'Demographics_% Non-Hispanic White':'%_Non-Hispanic_White',
         'Demographics_# Not Proficient in English':'#_Not_Proficient_in_English',
         'Demographics_% Not Proficient in English':'%_Not_Proficient_in_English',
         'Demographics_% Female':'%_Female',
         'Demographics_# Rural':'#_Pop_Living_in_Rural_Area',
         'Demographics_% Rural':'%_Rural'}, 
           inplace = True)
        
        # Drop unnecessary columns
        df2 = df2.drop(['Life expectancy_Life Expectancy',
         'Life expectancy_95% CI - Low',
         'Life expectancy_95% CI - High',
         'Life expectancy_Life Expectancy (Black)',
         'Life expectancy_Life Expectancy (Hispanic)',
         'Life expectancy_Life Expectancy (White)',
         'Premature age-adjusted mortality_95% CI - Low',
         'Premature age-adjusted mortality_95% CI - High',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Black)',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (Hispanic)',
         'Premature age-adjusted mortality_Age-Adjusted Mortality (White)',
         'Child mortality_95% CI - Low',
         'Child mortality_95% CI - High',
         'Child mortality_Child Mortality Rate (Black)',
         'Child mortality_Child Mortality Rate (Hispanic)',
         'Child mortality_Child Mortality Rate (White)',
         'Infant mortality_95% CI - Low',
         'Infant mortality_95% CI - High',
         'Infant mortality_Infant Mortality Rate (Black)',
         'Infant mortality_Infant Mortality Rate (Hispanic)',
         'Infant mortality_Infant Mortality Rate (White)',
         'Frequent physical distress_% Frequent Physical Distress',
         'Frequent physical distress_95% CI - Low',
         'Frequent physical distress_95% CI - High',
         'Frequent mental distress_% Frequent Mental Distress',
         'Frequent mental distress_95% CI - Low',
         'Frequent mental distress_95% CI - High',
         'Diabetes prevalence_95% CI - Low',
         'Diabetes prevalence_95% CI - High',
         'Motor vehicle crash deaths_95% CI - Low',
         'Motor vehicle crash deaths_95% CI - High',
         'Insufficient sleep_% Insufficient Sleep',
         'Insufficient sleep_95% CI - Low',
         'Insufficient sleep_95% CI - High',
         'Uninsured adults_95% CI - Low',
         'Uninsured adults_95% CI - High',
         'Uninsured children_95% CI - Low',
         'Uninsured children_95% CI - High',
         'Disconnected youth_% Disconnected Youth',
         'Median household income_95% CI - Low',
         'Median household income_95% CI - High',
         'Median household income_Household income (Black)',
         'Median household income_Household income (Hispanic)',
         'Median household income_Household income (White)',
         'Residential segregation - black/white_Segregation index',
         'Residential segregation - non-white/white_Segregation Index',
         'Homicides_95% CI - Low',
         'Homicides_95% CI - High',
         'Firearm fatalities_# Firearm Fatalities',
         'Firearm fatalities_Firearm Fatalities Rate',
         'Firearm fatalities_95% CI - Low',
         'Firearm fatalities_95% CI - High',
         'Homeownership_# Homeowners',
         'Homeownership_% Homeowners',
         'Homeownership_95% CI - Low',
         'Homeownership_95% CI - High',
         'Severe housing cost burden_# Households with Severe Cost Burden',
         'Severe housing cost burden_% Severe Housing Cost Burden',
         'Severe housing cost burden_95% CI - Low',
         'Severe housing cost burden_95% CI - High',
         'Demographics_95% CI - Low',
         'Demographics_95% CI - High']
                          ,axis=1)
        
        # Join two dataframes
        joined_df = pd.merge(df, df2, on='FIPS', how='outer')
        joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
        joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        joined_df['Year'] = '2019'
        
        # append to list
        appended_data2019.append(joined_df)
        
#convert to dataframe
appended_data2019 = pd.concat(appended_data2019)

2019 County Health Rankings Alabama Data - v1_0.xls
2019 County Health Rankings Alaska Data - v1_0.xls
2019 County Health Rankings Arizona Data - v1_0.xls
2019 County Health Rankings Arkansas Data - v1_0.xls
2019 County Health Rankings California Data - v1_0.xls
2019 County Health Rankings Colorado Data - v1_0.xls
2019 County Health Rankings Connecticut Data - v1_0.xls
2019 County Health Rankings Delaware Data - v1_0.xls
2019 County Health Rankings District of Columbia Data - v1_0.xls
2019 County Health Rankings Florida Data - v1_0.xls
2019 County Health Rankings Georgia Data - v1_0.xls
2019 County Health Rankings Hawaii Data - v1_0.xls
2019 County Health Rankings Idaho Data - v1_0.xls
2019 County Health Rankings Illinois Data - v1_0.xls
2019 County Health Rankings Indiana Data - v1_0.xls
2019 County Health Rankings Iowa Data - v1_0.xls
2019 County Health Rankings Kansas Data - v2.xls
2019 County Health Rankings Kentucky Data - v1_0.xls
2019 County Health Rankings Louisiana Data - v1_0

In [12]:
appended_data2019.head(20)

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,Avg_Days_Poor_Physical_Health_per_Month,Avg_Days_Mentally_Unhealthy_per_Month,Low_BirthWt_Unreliable,%_Low_Birthweight_Births,%_Adult_Smokers,...,#_Hispanic,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Year
0,1000,Alabama,NaN,9917.232898,21.402406,4.400458,4.577367,NaN,10.145493,21.538154,...,211058,4.329620,3196852,65.579855,48998,1.074846,51.590595,1957932.0,40.963183,2019
1,1001,Alabama,Autauga,8824.057123,18.411124,4.200578,4.306739,NaN,8.475719,19.124658,...,1586,2.857452,41336,74.473912,430,0.828548,51.342246,22921.0,42.002162,2019
2,1003,Alabama,Baldwin,7224.632160,18.060458,4.098748,4.249649,NaN,8.338683,16.795485,...,9675,4.550200,176582,83.047388,872,0.454512,51.452772,77060.0,42.279099,2019
3,1005,Alabama,Barbour,9586.165037,25.773416,5.067438,4.634994,NaN,10.952624,21.540878,...,1063,4.206569,11613,45.955679,297,1.199273,47.229917,18613.0,67.789635,2019
4,1007,Alabama,Bibb,11783.543675,19.996912,4.363377,4.315710,NaN,11.105003,19.916404,...,598,2.638080,16842,74.298571,84,0.394348,46.453150,15663.0,68.352607,2019
5,1009,Alabama,Blount,10908.101822,21.095323,4.512753,4.701599,NaN,7.926829,19.652158,...,5549,9.565097,50439,86.944306,1013,1.870004,50.688639,51562.0,89.951502,2019
6,1011,Alabama,Bullock,12066.910263,29.407714,5.226619,4.791918,NaN,13.732004,23.478291,...,850,8.245223,2196,21.301775,109,1.104692,45.533029,5607.0,51.374382,2019
7,1013,Alabama,Butler,14018.606262,26.502613,5.117433,5.134573,NaN,12.411765,22.197656,...,285,1.437579,10229,51.596469,135,0.712702,53.432535,14921.0,71.232157,2019
8,1015,Alabama,Calhoun,12217.759984,19.584701,4.290448,4.476572,NaN,9.002330,20.462232,...,4302,3.749739,82961,72.311031,960,0.880968,51.930653,39955.0,33.696826,2019
9,1017,Alabama,Chambers,11273.170980,25.219836,5.037045,4.932646,NaN,12.446043,22.040370,...,822,2.438229,18710,55.497879,49,0.153894,52.125293,16816.0,49.148034,2019


In [13]:
list(appended_data2019.columns.values)

['FIPS',
 'State',
 'County/Borough/Parish',
 'Yrs_Potential_Life_Lost_per100K(YPLL)',
 '%_Fair/Poor_Health',
 'Avg_Days_Poor_Physical_Health_per_Month',
 'Avg_Days_Mentally_Unhealthy_per_Month',
 'Low_BirthWt_Unreliable',
 '%_Low_Birthweight_Births',
 '%_Adult_Smokers',
 '%_Adult_Obesity',
 'Healthy_Food_Access(0=Worst,10=Best)',
 '%_Adults_Physical_Inactivity',
 '%_Access_to_Excercise_Opportunities',
 '%_Adults_Excessive_Drinking',
 '#_Alcohol-Impaired_Motor_Vehicle_Deaths',
 '#_Motor_Vehicle_Deaths_x',
 '%_Driving_Deaths_with_Alcohol_Involvement',
 '#_Chlamydia_Cases',
 'Chlamydia_Cases_per_100K',
 'Teen_Birth_Rate',
 '#_Ppl_Under65_w/o_Insurance',
 '%_Ppl_Under65_w/o_Insurance',
 '#_Primary_Care_Physicians(PCP)_in_Pt_Care',
 'PCP_per_100K',
 'Population_to_PCP_ratio',
 '#_Dentists',
 'Dentists_per_100K',
 'Population_to_Dentists_ratio',
 '#_Mental_Health_Providers(MHP)',
 'MHP_per_100K',
 'Population_to_MHP_ratio',
 'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
 '

In [14]:
appended_data2019.count()

FIPS                                                    3077
State                                                   3077
County/Borough/Parish                                   3027
Yrs_Potential_Life_Lost_per100K(YPLL)                   2845
%_Fair/Poor_Health                                      3077
Avg_Days_Poor_Physical_Health_per_Month                 3077
Avg_Days_Mentally_Unhealthy_per_Month                   3077
Low_BirthWt_Unreliable                                   166
%_Low_Birthweight_Births                                2970
%_Adult_Smokers                                         3077
%_Adult_Obesity                                         3077
Healthy_Food_Access(0=Worst,10=Best)                    3058
%_Adults_Physical_Inactivity                            3077
%_Access_to_Excercise_Opportunities                     3071
%_Adults_Excessive_Drinking                             3077
#_Alcohol-Impaired_Motor_Vehicle_Deaths                 3044
#_Motor_Vehicle_Deaths_x

In [15]:
appended_data2019.isnull().any()

FIPS                                                    False
State                                                   False
County/Borough/Parish                                    True
Yrs_Potential_Life_Lost_per100K(YPLL)                    True
%_Fair/Poor_Health                                      False
Avg_Days_Poor_Physical_Health_per_Month                 False
Avg_Days_Mentally_Unhealthy_per_Month                   False
Low_BirthWt_Unreliable                                   True
%_Low_Birthweight_Births                                 True
%_Adult_Smokers                                         False
%_Adult_Obesity                                         False
Healthy_Food_Access(0=Worst,10=Best)                     True
%_Adults_Physical_Inactivity                            False
%_Access_to_Excercise_Opportunities                      True
%_Adults_Excessive_Drinking                             False
#_Alcohol-Impaired_Motor_Vehicle_Deaths                  True
#_Motor_

In [17]:
appended_data2019.loc[appended_data2019.isnull().any(axis=1)]

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,Avg_Days_Poor_Physical_Health_per_Month,Avg_Days_Mentally_Unhealthy_per_Month,Low_BirthWt_Unreliable,%_Low_Birthweight_Births,%_Adult_Smokers,...,#_Hispanic,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Year
0,1000,Alabama,NaN,9917.232898,21.402406,4.400458,4.577367,NaN,10.145493,21.538154,...,211058,4.329620,3196852,65.579855,48998,1.074846,51.590595,1957932.0,40.963183,2019
1,1001,Alabama,Autauga,8824.057123,18.411124,4.200578,4.306739,NaN,8.475719,19.124658,...,1586,2.857452,41336,74.473912,430,0.828548,51.342246,22921.0,42.002162,2019
2,1003,Alabama,Baldwin,7224.632160,18.060458,4.098748,4.249649,NaN,8.338683,16.795485,...,9675,4.550200,176582,83.047388,872,0.454512,51.452772,77060.0,42.279099,2019
3,1005,Alabama,Barbour,9586.165037,25.773416,5.067438,4.634994,NaN,10.952624,21.540878,...,1063,4.206569,11613,45.955679,297,1.199273,47.229917,18613.0,67.789635,2019
4,1007,Alabama,Bibb,11783.543675,19.996912,4.363377,4.315710,NaN,11.105003,19.916404,...,598,2.638080,16842,74.298571,84,0.394348,46.453150,15663.0,68.352607,2019
5,1009,Alabama,Blount,10908.101822,21.095323,4.512753,4.701599,NaN,7.926829,19.652158,...,5549,9.565097,50439,86.944306,1013,1.870004,50.688639,51562.0,89.951502,2019
6,1011,Alabama,Bullock,12066.910263,29.407714,5.226619,4.791918,NaN,13.732004,23.478291,...,850,8.245223,2196,21.301775,109,1.104692,45.533029,5607.0,51.374382,2019
7,1013,Alabama,Butler,14018.606262,26.502613,5.117433,5.134573,NaN,12.411765,22.197656,...,285,1.437579,10229,51.596469,135,0.712702,53.432535,14921.0,71.232157,2019
8,1015,Alabama,Calhoun,12217.759984,19.584701,4.290448,4.476572,NaN,9.002330,20.462232,...,4302,3.749739,82961,72.311031,960,0.880968,51.930653,39955.0,33.696826,2019
9,1017,Alabama,Chambers,11273.170980,25.219836,5.037045,4.932646,NaN,12.446043,22.040370,...,822,2.438229,18710,55.497879,49,0.153894,52.125293,16816.0,49.148034,2019


In [18]:
appended_data2019 = appended_data2019.fillna(value=0)
appended_data2019.head(20)

,FIPS,State,County/Borough/Parish,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,Avg_Days_Poor_Physical_Health_per_Month,Avg_Days_Mentally_Unhealthy_per_Month,Low_BirthWt_Unreliable,%_Low_Birthweight_Births,%_Adult_Smokers,...,#_Hispanic,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Year
0,1000,Alabama,0,9917.232898,21.402406,4.400458,4.577367,0,10.145493,21.538154,...,211058,4.329620,3196852,65.579855,48998,1.074846,51.590595,1957932.0,40.963183,2019
1,1001,Alabama,Autauga,8824.057123,18.411124,4.200578,4.306739,0,8.475719,19.124658,...,1586,2.857452,41336,74.473912,430,0.828548,51.342246,22921.0,42.002162,2019
2,1003,Alabama,Baldwin,7224.632160,18.060458,4.098748,4.249649,0,8.338683,16.795485,...,9675,4.550200,176582,83.047388,872,0.454512,51.452772,77060.0,42.279099,2019
3,1005,Alabama,Barbour,9586.165037,25.773416,5.067438,4.634994,0,10.952624,21.540878,...,1063,4.206569,11613,45.955679,297,1.199273,47.229917,18613.0,67.789635,2019
4,1007,Alabama,Bibb,11783.543675,19.996912,4.363377,4.315710,0,11.105003,19.916404,...,598,2.638080,16842,74.298571,84,0.394348,46.453150,15663.0,68.352607,2019
5,1009,Alabama,Blount,10908.101822,21.095323,4.512753,4.701599,0,7.926829,19.652158,...,5549,9.565097,50439,86.944306,1013,1.870004,50.688639,51562.0,89.951502,2019
6,1011,Alabama,Bullock,12066.910263,29.407714,5.226619,4.791918,0,13.732004,23.478291,...,850,8.245223,2196,21.301775,109,1.104692,45.533029,5607.0,51.374382,2019
7,1013,Alabama,Butler,14018.606262,26.502613,5.117433,5.134573,0,12.411765,22.197656,...,285,1.437579,10229,51.596469,135,0.712702,53.432535,14921.0,71.232157,2019
8,1015,Alabama,Calhoun,12217.759984,19.584701,4.290448,4.476572,0,9.002330,20.462232,...,4302,3.749739,82961,72.311031,960,0.880968,51.930653,39955.0,33.696826,2019
9,1017,Alabama,Chambers,11273.170980,25.219836,5.037045,4.932646,0,12.446043,22.040370,...,822,2.438229,18710,55.497879,49,0.153894,52.125293,16816.0,49.148034,2019


In [19]:
appended_data2019.to_csv(os.path.join("Cleaned_Files","2019_County_Health_Ranking_Data_Test.csv"),index=False)

In [ ]:
appended_data2018 = []
path = os.path.join("Original_Files","CountyHealthRankingsData","2018")
for WorkingFile in os.listdir(path):
        #identifies the list of files to be grouped together from designated folder
        print(WorkingFile)
        
        # Import the excel file by joining the path directory and file name together and call Ranked Data and Additonal Measure tabs
        file = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Ranked Measure Data',header=(0,1))
        file2 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Additional Measure Data',header=(0,1))
       
        # Load the excel files as two dataframes
        df = pd.DataFrame(file)
        df2 = pd.DataFrame(file2)
        
        # Code from Data_cleaning
       
        # Clean Ranked Data First by addressing multiindex column names
        df.columns = df.columns.map('_'.join)
        
        # Rename Columns
       
        
        # Drop unnecessary columns

        
        # Clean Additional Data Second by addressing multiindex column names
        df2.columns = df2.columns.map('_'.join)
        
        # Rename Columns

        
        # Drop unnecessary columns

        
        # Join two dataframes
        joined_df = pd.merge(df, df2, on='FIPS', how='outer')
        joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
        joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        joined_df['Year'] = '2018'
        
        # append to list
        appended_data2018.append(joined_df)
        
#convert to dataframe
appended_data2018 = pd.concat(appended_data2018)

In [ ]:
appended_data2018.head(20)

In [ ]:
list(appended_data2018.columns.values)

In [ ]:
appended_data2018.count()

In [ ]:
appended_data2018.isnull().any()

In [ ]:
appended_data2018.loc[appended_data2018.isnull().any(axis=1)]

In [ ]:
appended_data2018 = appended_data2018.fillna(value=0)
appended_data2018.head(20)

In [ ]:
appended_data2018.to_csv(os.path.join("Cleaned_Files","2018_County_Health_Ranking_Data_Test.csv"),index=False)

In [ ]:
appended_data2017 = []
path = os.path.join("Original_Files","CountyHealthRankingsData","2017")
for WorkingFile in os.listdir(path):
        #identifies the list of files to be grouped together from designated folder
        print(WorkingFile)
        
        # Import the excel file by joining the path directory and file name together and call Ranked Data and Additonal Measure tabs
        file = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Ranked Measure Data',header=(0,1))
        file2 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Additional Measure Data',header=(0,1))
       
        # Load the excel files as two dataframes
        df = pd.DataFrame(file)
        df2 = pd.DataFrame(file2)
        
        # Code from Data_cleaning
       
        # Clean Ranked Data First by addressing multiindex column names
        df.columns = df.columns.map('_'.join)
        
        # Rename Columns
       
        
        # Drop unnecessary columns

        
        # Clean Additional Data Second by addressing multiindex column names
        df2.columns = df2.columns.map('_'.join)
        
        # Rename Columns

        
        # Drop unnecessary columns

        
        # Join two dataframes
        joined_df = pd.merge(df, df2, on='FIPS', how='outer')
        joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
        joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        joined_df['Year'] = '2017'
        
        # append to list
        appended_data2018.append(joined_df)
        
#convert to dataframe
appended_data2017 = pd.concat(appended_data2017)

In [ ]:
appended_data2017.head(20)

In [ ]:
list(appended_data2017.columns.values)

In [ ]:
appended_data2017.count()

In [ ]:
appended_data2017.isnull().any()

In [ ]:
appended_data2017.loc[appended_data2017.isnull().any(axis=1)]

In [ ]:
appended_data2017 = appended_data2017.fillna(value=0)
appended_data2017.head(20)

In [ ]:
appended_data2017.to_csv(os.path.join("Cleaned_Files","2017_County_Health_Ranking_Data_Test.csv"),index=False)

In [ ]:
appended_data2016 = []
path = os.path.join("Original_Files","CountyHealthRankingsData","2016")
for WorkingFile in os.listdir(path):
        #identifies the list of files to be grouped together from designated folder
        print(WorkingFile)
        
        # Import the excel file by joining the path directory and file name together and call Ranked Data and Additonal Measure tabs
        file = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Ranked Measure Data',header=(0,1))
        file2 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Additional Measure Data',header=(0,1))
       
        # Load the excel files as two dataframes
        df = pd.DataFrame(file)
        df2 = pd.DataFrame(file2)
        
        # Code from Data_cleaning
       
        # Clean Ranked Data First by addressing multiindex column names
        df.columns = df.columns.map('_'.join)
        
        # Rename Columns
       
        
        # Drop unnecessary columns

        
        # Clean Additional Data Second by addressing multiindex column names
        df2.columns = df2.columns.map('_'.join)
        
        # Rename Columns

        
        # Drop unnecessary columns

        
        # Join two dataframes
        joined_df = pd.merge(df, df2, on='FIPS', how='outer')
        joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
        joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        joined_df['Year'] = '2016'
        
        # append to list
        appended_data2016.append(joined_df)
        
#convert to dataframe
appended_data2016 = pd.concat(appended_data2016)

In [ ]:
appended_data2016.head(20)

In [ ]:
list(appended_data2016.columns.values)

In [ ]:
appended_data2016.count()

In [ ]:
appended_data2016.isnull().any()

In [ ]:
appended_data2016.loc[appended_data2016.isnull().any(axis=1)]

In [ ]:
appended_data2016 = appended_data2016.fillna(value=0)
appended_data2016.head(20)

In [ ]:
appended_data2016.to_csv(os.path.join("Cleaned_Files","2016_County_Health_Ranking_Data_Test.csv"),index=False)

In [ ]:
appended_data2015 = []
path = os.path.join("Original_Files","CountyHealthRankingsData","2015")
for WorkingFile in os.listdir(path):
        #identifies the list of files to be grouped together from designated folder
        print(WorkingFile)
        
        # Import the excel file by joining the path directory and file name together and call Ranked Data and Additonal Measure tabs
        file = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Ranked Measure Data',header=(0,1))
        file2 = pd.read_excel(os.path.join(path, WorkingFile),sheet_name='Additional Measure Data',header=(0,1))
       
        # Load the excel files as two dataframes
        df = pd.DataFrame(file)
        df2 = pd.DataFrame(file2)
        
        # Code from Data_cleaning
       
        # Clean Ranked Data First by addressing multiindex column names
        df.columns = df.columns.map('_'.join)
        
        # Rename Columns
       
        
        # Drop unnecessary columns

        
        # Clean Additional Data Second by addressing multiindex column names
        df2.columns = df2.columns.map('_'.join)
        
        # Rename Columns

        
        # Drop unnecessary columns

        
        # Join two dataframes
        joined_df = pd.merge(df, df2, on='FIPS', how='outer')
        joined_df = joined_df.drop(['State_y','County/Borough/Parish_y'],axis=1)
        joined_df.rename(columns = {'State_x':'State','County/Borough/Parish_x':'County/Borough/Parish'},inplace = True)
        joined_df['Year'] = '2015'
        
        # append to list
        appended_data2015.append(joined_df)
        
#convert to dataframe
appended_data2015 = pd.concat(appended_data2015)

In [ ]:
appended_data2015.head(20)

In [ ]:
list(appended_data2015.columns.values)

In [ ]:
appended_data2015.count()

In [ ]:
appended_data2015.isnull().any()

In [ ]:
appended_data2015.loc[appended_data2015.isnull().any(axis=1)]

In [ ]:
appended_data2015 = appended_data2015.fillna(value=0)
appended_data2015.head(20)

In [ ]:
appended_data2015.to_csv(os.path.join("Cleaned_Files","2015_County_Health_Ranking_Data_Test.csv"),index=False)